In [34]:
import pandas as pd

In [35]:
df = pd.read_csv('../data/lstm_dataset_final.csv')

df = df.sort_values(by=['playerId', 'season'])

features = [
    'xGoals', 'pAssists', 'sAssists', 'sog', 'points', 'goals',
    'on_ice_chances', 'on_ice_goals', 'hits', 'penality_mins', 'pen_drawn',
    'corsi', 'xGoalsForAfterShifts', 'corsiForAfterShifts',
    'icetime_per_game', 'shot_percentage', 'points_per_60',
    'ixG-goals', 'ppg', 'apg', 'gpg',
    'xGoalsFor-goalsFor_team', 'age', 'age2', 'games_played_per',
    'pos_C', 'pos_D', 'pos_L', 'pos_R'
]
targets = ['next_games_played_per', 'next_goals_per_game', 'next_assists_per_game']

df = df.dropna(subset=features + targets)

for col in ['pos_C', 'pos_D', 'pos_L', 'pos_R']:
    df[col] = df[col].astype(int)
    
df.head()


,Unnamed: 0.1,Unnamed: 0,playerId,season,name,position,team,games_played,xGoals,pAssists,...,xGoalsFor-goalsFor_team,age2,games_played_per,pos_C,pos_D,pos_L,pos_R,next_games_played_per,next_goals_per_game,next_assists_per_game
27,27,725,8448208,2015,Jaromir Jagr,R,FLA,79,19.76,21.0,...,-8.23,1849,0.963415,0,0,0,1,0.987805,0.197531,0.370370
1516,1516,1652,8448208,2016,Jaromir Jagr,R,FLA,81,23.04,20.0,...,20.95,1936,0.987805,0,0,0,1,0.268293,0.045455,0.272727
840,840,137,8462038,2015,Shane Doan,R,ARI,72,21.16,12.0,...,-6.89,1521,0.878049,0,0,0,1,0.890244,0.068493,0.287671
129,129,484,8462042,2015,Jarome Iginla,R,COL,82,18.66,16.0,...,-10.79,1444,1.000000,0,0,0,1,0.963415,0.177215,0.164557
437,437,173,8464989,2015,Matt Cullen,C,PIT,82,11.16,11.0,...,-0.07,1521,1.000000,1,0,0,0,0.878049,0.180556,0.250000


In [36]:
from collections import defaultdict
import numpy as np

In [ ]:
seq = 3

X = []
y = []

player_groups = df.groupby('playerId')

In [5]:
from sklearn.preprocessing import StandardScaler

In [6]:
X = df[features].values
y = df[targets].values

In [7]:
X_scaler = StandardScaler()
y_scaler = StandardScaler()

X_scaled = X_scaler.fit_transform(X)
y_scaled = y_scaler.fit_transform(y)

X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

In [8]:
from torch.utils.data import Dataset

In [9]:
class NHLDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [10]:
import torch.nn as nn

In [11]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers,
                            batch_first=True, dropout=dropout if num_layers > 1 else 0.0)
        self.fc = nn.Linear(hidden_size, 3)

    def forward(self, x):
        _, (hn, _) = self.lstm(x)
        out = self.fc(hn[-1])
        return out

In [17]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import torch

In [18]:
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

train_loader = DataLoader(NHLDataset(X_train, y_train), batch_size=64, shuffle=True)
val_loader = DataLoader(NHLDataset(X_val, y_val), batch_size=64)

In [19]:
import optuna

/Users/ryanvu/.pyenv/versions/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
def objective(trial):
    hidden_size = trial.suggest_int("hidden_size", 64, 256)
    num_layers = trial.suggest_int("num_layers", 1, 3)
    dropout = trial.suggest_float("dropout", 0.0, 0.5)
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    batch_size = 64

    model = LSTM(input_size=X_train.shape[2], hidden_size=hidden_size,
                      num_layers=num_layers, dropout=dropout)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()

    for epoch in range(10):
        model.train()
        for xb, yb in train_loader:
            pred = model(xb)
            loss = loss_fn(pred, yb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    model.eval()
    preds, targets = [], []
    with torch.no_grad():
        for xb, yb in val_loader:
            out = model(xb)
            preds.append(out.numpy())
            targets.append(yb.numpy())

    preds = np.concatenate(preds, axis=0)
    targets = np.concatenate(targets, axis=0)

    preds_orig = y_scaler.inverse_transform(preds)
    targets_orig = y_scaler.inverse_transform(targets)

    return r2_score(targets_orig, preds_orig)

In [119]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

[I 2025-06-22 23:29:24,971] A new study created in memory with name: no-name-2976a02e-e42c-4476-8fbf-8dde95bdd989
[I 2025-06-22 23:29:28,237] Trial 0 finished with value: 0.5367634892463684 and parameters: {'hidden_size': 190, 'num_layers': 1, 'dropout': 0.054696474651310556, 'lr': 0.0064808578814854285}. Best is trial 0 with value: 0.5367634892463684.
[I 2025-06-22 23:29:34,680] Trial 1 finished with value: 0.5339212417602539 and parameters: {'hidden_size': 191, 'num_layers': 2, 'dropout': 0.02066519600246991, 'lr': 0.0014777671238253094}. Best is trial 0 with value: 0.5367634892463684.
[I 2025-06-22 23:29:37,823] Trial 2 finished with value: 0.5365378260612488 and parameters: {'hidden_size': 166, 'num_layers': 1, 'dropout': 0.18954468729874924, 'lr': 0.002386972010702318}. Best is trial 0 with value: 0.5367634892463684.
[I 2025-06-22 23:29:39,901] Trial 3 finished with value: 0.5085695385932922 and parameters: {'hidden_size': 74, 'num_layers': 1, 'dropout': 0.014457983249435702, 'lr'

In [120]:
best_params = study.best_trial.params
print(best_params)

{'hidden_size': 208, 'num_layers': 2, 'dropout': 0.21265886609338036, 'lr': 0.0005223643206310589}


In [121]:
best_model = LSTM(
    input_size=X_train.shape[2],
    hidden_size=best_params["hidden_size"],
    num_layers=best_params["num_layers"],
    dropout=best_params["dropout"] if best_params["num_layers"] > 1 else 0.0
)

optimizer = torch.optim.Adam(best_model.parameters(), lr=best_params["lr"])
loss_fn = nn.MSELoss()

for epoch in range(30):
    best_model.train()
    for xb, yb in train_loader:
        pred = best_model(xb)
        loss = loss_fn(pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if epoch % 5 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

Epoch 0, Loss: 0.3195
Epoch 5, Loss: 0.3988
Epoch 10, Loss: 0.4585
Epoch 15, Loss: 0.3510
Epoch 20, Loss: 0.4351
Epoch 25, Loss: 0.4418


In [122]:
best_model.eval()
preds, targets = [], []

with torch.no_grad():
    for xb, yb in val_loader:
        out = best_model(xb)
        preds.append(out.numpy())
        targets.append(yb.numpy())

preds = np.concatenate(preds, axis=0)
targets = np.concatenate(targets, axis=0)

preds_orig = y_scaler.inverse_transform(preds)
targets_orig = y_scaler.inverse_transform(targets)

In [123]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

print("R² Score:", r2_score(targets_orig, preds_orig))
print("MAE:", mean_absolute_error(targets_orig, preds_orig))
print("MSE:", mean_squared_error(targets_orig, preds_orig))
print("RMSE:", np.sqrt(mean_squared_error(targets_orig, preds_orig)))

R² Score: 0.5390410423278809
MAE: 0.11306972056627274
MSE: 0.028025291860103607
RMSE: 0.16740756213535757


In [124]:
target_names = ['next_games_played_per', 'next_goals_per_game', 'next_assists_per_game']
for i, name in enumerate(target_names):
    print(f"{name}")
    print("R²:", r2_score(targets_orig[:, i], preds_orig[:, i]))
    print("MAE:", mean_absolute_error(targets_orig[:, i], preds_orig[:, i]))
    print("MSE:", mean_squared_error(targets_orig[:, i], preds_orig[:, i]))
    print("RMSE:", np.sqrt(mean_squared_error(targets_orig[:, i], preds_orig[:, i])))

next_games_played_per
R²: 0.4227102994918823
MAE: 0.1939525604248047
MSE: 0.06216659024357796
RMSE: 0.24933228881069125
next_goals_per_game
R²: 0.5904867649078369
MAE: 0.057645585387945175
MSE: 0.007465627510100603
RMSE: 0.08640386281932425
next_assists_per_game
R²: 0.6039255857467651
MAE: 0.08761127293109894
MSE: 0.014443689957261086
RMSE: 0.12018190361806176


In [125]:
import joblib

In [127]:
torch.save(best_model.state_dict(), "../models/lstm_nhl_model.pt")
joblib.dump(X_scaler, "../models/X_scaler.pkl")
joblib.dump(y_scaler, "../models/y_scaler.pkl")

['../models/y_scaler.pkl']